# ccdc_year_stack.ipynb
#### Dave McCaffrey — Jan 31, 2021

This script reformats the output of the Google Earth Enigne implementation of the CCDC temporal change detection algorithm. The GEE CCDC scripts outputs a mutli-band raster containing temporal breaks for each pixel. A given pixel may have several breaks in a timeseries, so band 1 equals the first disturbance, band 2 is the second...  

This script reformats the multi-band tBreak output from GEE and creates a multi-band geotiff wherein each band shows all disturbances within a given calendar year. For an example landsat timeseries, band 1 = all distrubances in 1985, band 2 = 1986...

Date values are stored as fractions of a year. In 2021, July 1st is the 182nd day of the year. 182 /365 = 0.4986... thus a change registered on July 1, 2021 would be stored as 2021.4986 . I am unsure if the CCDC output accounts for leap years. Due to the initialization period for the CCDC having different outlier removal among pixels, these dates should be considered and approximation, and not exact. Temporal edge effects suggest that the first and last year in the timeseries should be omitted from analysis. 

Only the max date value in a given year is stored, however there should not be multiple disturbances in a calendar year. Each initialization period requires a 12 image series, and each disturbance requires 6 consectuive anomalous images. 18 images x 16 revisit = 288 days. The only way to get 2 disturbances in the same calendar year is if they both happen in snowy months... so these should not be considered valid disturbances. 

The output 'year stack' is formated to the same resoltuion, extent, and projection as the CCDC file input. 

Both the input and output filepath must be manually specified. 

In [3]:
import numpy as np 
# import gdal
# import osr
from osgeo import gdal
from osgeo import osr

### Enter filepath to GEE CCDC break file

In [4]:
file = r"C:\Users\owner\Dropbox\lc_contract\Data\CCDC\ls58_allbands_20210329\seven_band_tBreak (34).tif" # output from GEE CCDC 

ds = gdal.Open(file)
arr = ds.ReadAsArray()

In [5]:
# apply probability treshold
prob_file = r"C:\Users\owner\Dropbox\lc_contract\Data\CCDC\ls58_allbands_20210329\seven_band_changeProb (23).tif"
data = gdal.Open(prob_file)
arr2 = data.ReadAsArray()
arr3 = np.where(arr2 < 0.5, 0, 1)
arr = arr * arr3

In [6]:
# make destination 
out_path = r"C:\Users\owner\Dropbox\lc_contract\Data\CCDC\ls58_allbands_20210329\tbreak_prob_mask.tif"

dst_ds = gdal.GetDriverByName('GTiff').Create(out_path, 
                                              arr.shape[2], 
                                              arr.shape[1], 
                                              arr.shape[0], 
                                              gdal.GDT_Float32)

# get ans set geotransform
geo_trans = ds.GetGeoTransform() #from ds in cell 2 
dst_ds.SetGeoTransform(geo_trans)

# get and set ESPG 
proj = osr.SpatialReference(wkt=ds.GetProjection()) #from ds in cell 2 
srs = osr.SpatialReference()            
srs.ImportFromEPSG(int(proj.GetAttrValue('AUTHORITY',1))) # ESPG from proj 
dst_ds.SetProjection(srs.ExportToWkt()) 

# loop through array, new band for each year in stack 
for i in np.arange(1,arr.shape[0]+1):
    dst_ds.GetRasterBand(int(i)).WriteArray(arr[i-1])  
    
# write to disk
dst_ds.FlushCache()                     
dst_ds = None

This cell calculates the min and max disturbance year and creates an array of all years between, inclusive of the start and stop year.

In [7]:
year_series = np.arange(np.floor(np.nanmin(arr[np.nonzero(arr)])), np.floor(np.nanmax(arr[np.nonzero(arr)])) + 1 ).astype(int).tolist()

This cell iterates through the years, masks invalid data in the tbreak array, removes all values less than the target year and greater than or equal to the target year plus one, then collapses all values in the year range into a single 2d array, stored in a list. The list of 2d arrays is stacked to make a 3d array. 

In [8]:
array_list = []

for year in year_series:
    
    arr_valid = np.ma.masked_invalid(arr)
       
    arr_lt = np.ma.masked_where(arr_valid < year, arr_valid) 
    arr_gte = np.ma.masked_where(arr_lt >= year+1, arr_lt)
    arr_max = arr_gte.max(axis=0)
    
    array_list.append(arr_max.filled(fill_value = 0.0)) 

array_3d = np.stack(array_list, axis=0)   

### Enter Output filepath

The geotiff parameters are copied from the input image, and the 3d array is stored in the new geotiff and exported. 

In [9]:
# make destination 
out_path =  r"C:\Users\owner\Dropbox\lc_contract\Data\CCDC\ls58_allbands_20210329\year_prob_stack.tif"

dst_ds = gdal.GetDriverByName('GTiff').Create(out_path, 
                                              array_3d.shape[2], 
                                              array_3d.shape[1], 
                                              array_3d.shape[0], 
                                              gdal.GDT_Float32)

# get ans set geotransform
geo_trans = ds.GetGeoTransform() #from ds in cell 2 
dst_ds.SetGeoTransform(geo_trans)

# get and set ESPG 
proj = osr.SpatialReference(wkt=ds.GetProjection()) #from ds in cell 2 
srs = osr.SpatialReference()            
srs.ImportFromEPSG(int(proj.GetAttrValue('AUTHORITY',1))) # ESPG from proj 
dst_ds.SetProjection(srs.ExportToWkt()) 

# loop through array, new band for each year in stack 
for i in np.arange(1,array_3d.shape[0]+1):
    dst_ds.GetRasterBand(int(i)).WriteArray(array_3d[i-1])  
    
# write to disk
dst_ds.FlushCache()                     
dst_ds = None